# Creating question variants
EDSL comes with a variety of features for efficiently generating different versions of questions in surveys. This notebook demonstrates methods for doing this with `Scenario` objects.

## What is a `Scenario`?
A `Scenario` is a dictionary of one or more key/value pairs representing data or content to be added to questions; a `ScenarioList` is a list of `Scenario` objects. 
Scenario keys are used as question parameters that get replaced with the values when the scenarios are added to the questions, allowing you to create variants of questions efficiently.
For example:

```
from edsl import QuestionFreeText, ScenarioList, Scenario

q = QuestionFreeText(
    question_name = "favorite",
    question_text = "What is your favorite {{ scenario.thing }}?"
)

s = ScenarioList(
    Scenario({"thing": t}) for t in ["flower", "pizza topping", "chatbot"]
)
```

## Using scenarios
Scenarios can be added to questions when constructing a survey or when running it.
Functionally, the same question context is delivered to agents and models whether they are added during or after survey construction. 
The difference is how the information is arranged in the results that are generated by the models. 

## Methods

### Adding scenarios at survey construction: `loop`
Each question type (`QuestionMultipleChoice`, `QuestionFreeText`, etc.) has a `loop()` method that generates a copy of the question for each scenario in a `ScenarioList` that is passed to it, returning a list of the questions that are generated. 
The `loop()` method is used when survey questions are being constructed. 
The typical workflow is:

* Construct a (single) `Question` with one or more parameters
* Construct a `ScenarioList`
* Call the `loop()` method on the question and pass it the scenario list
* Pass the list of the questions to a `Survey`

From the example above this looks like:
```
from edsl import Survey

questions = q.loop(s)
survey = Survey(questions)
results = survey.run()
```

When the survey is run, the results that are generated will include columns for each question and answer; there are no `scenario` columns in the results (unless scenarios are *also* added when the survey is run).

### Running a survey with scenarios: `by` 
Scenarios can also be passed to a question or survey at the time that it is run. 
This is done by calling the `by()` method on a survey, passing it the scenarios, and then calling the `run()` method.
The typical workflow is:

* Construct a question (or survey of multiple questions) with one or more parameters
* Construct scenarios
* Call the `by()` method on the question or survey and pass it the scenarios
* Call the `run()` method to administer the question or survey

From the example above this looks like:
```
results = q.by(s).run()
```

(If any agents or models have been created and specified, they would also be added in separated `by()` calls. 
See details on [designing agents](https://docs.expectedparrot.com/en/latest/agents.html) and [selecting language models](https://docs.expectedparrot.com/en/latest/language_models.html).)

## Example: Looping a question with scenarios
The `loop()` method is called on a `Question` object, and takes a `ScenarioList` of values to be inserted in copies of the question. 
We can optionally use the scenario key in the question name as well (so long as it is Pythonic); otherwise, unique identifiers are added to the original question name.

We start by constructing a question that takes a parameter:

In [1]:
from edsl import QuestionFreeText

q = QuestionFreeText(
    question_name = "features",
    question_text = "Briefly describe the features of this sailboat model: {{ scenario.sailboat_model }}"
)

Next we create a scenario list to pass to the `loop()` method. 
EDSL comes with many methods for generating scenarios from different data sources, such as PDF, PNG, CSV, docs, tables, dicts, lists, etc.
We can use the `from_source()` method to construct a scenario list and specify the data type ("csv", "pdf", "list", etc.). 
[Learn about other methods for generating scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html).

In [2]:
from edsl import ScenarioList

s = ScenarioList.from_source("list", "sailboat_model", ['Laser', 'Sunfish', 'Optimist', 'Finn'])
s

,sailboat_model
0,Laser
1,Sunfish
2,Optimist
3,Finn


Next we call the `loop()` method with the scenario list to create a list of the copies of the question, and verify that formatted questions have been generated:

In [3]:
questions = q.loop(s)
questions

[Question('free_text', question_name = """features_0""", question_text = """Briefly describe the features of this sailboat model: Laser"""),
 Question('free_text', question_name = """features_1""", question_text = """Briefly describe the features of this sailboat model: Sunfish"""),
 Question('free_text', question_name = """features_2""", question_text = """Briefly describe the features of this sailboat model: Optimist"""),
 Question('free_text', question_name = """features_3""", question_text = """Briefly describe the features of this sailboat model: Finn""")]

We can pass the questions to a `Survey` and then run it:

In [4]:
from edsl import Survey

survey = Survey(questions = questions)

results = survey.run()

We can check the columns of dataset of `Results` that have been generated, and see that there are sets of columns for each question identifiable by question name (but no `scenario` columns):

In [5]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.features_0
4,answer.features_1
5,answer.features_2
6,answer.features_3
7,cache_keys.features_0_cache_key
8,cache_keys.features_1_cache_key
9,cache_keys.features_2_cache_key


We can access [built-in methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html), e.g., printing a table:

In [6]:
results.select("answer.*")

,answer.features_1,answer.features_2,answer.features_3,answer.features_0
0,"The Sunfish is a popular and iconic sailboat known for its simplicity and ease of use, making it ideal for beginners and recreational sailing. Key features of the Sunfish include: 1. **Design**: The Sunfish has a distinctive, lightweight, and flat-bottomed hull design, which contributes to its stability and ease of handling. It typically measures about 13.9 feet in length and has a beam (width) of around 4.1 feet. 2. **Rigging**: It features a lateen rig, which comprises a single, triangular sail mounted on a simple, two-piece mast and boom. The lateen sail is easy to rig and adjust, making it accessible for novice sailors. 3. **Construction**: Traditionally, Sunfish sailboats were made of fiberglass, which provides durability and low maintenance. The hull is designed to be unsinkable, with foam flotation built in. 4. **Portability**: The boat is lightweight, generally around 120 pounds, which makes it easy to transport on a trailer or car top and launch from a beach or dock. 5. **Performance**: While designed for simplicity, the Sunfish can be quite fast and responsive, offering a fun sailing experience. It performs well in a variety of wind conditions. 6. **Popularity**: Due to its affordability, ease of use, and widespread availability, the Sunfish is one of the most popular sailboats in the world, with a strong community and numerous racing events dedicated to it. Overall, the Sunfish is celebrated for its versatility, making it a great choice for both casual sailing and competitive racing.","The Optimist sailboat is a small, single-handed dinghy designed specifically for young sailors. Here are some of its key features: 1. **Size and Design**: The Optimist is a pram dinghy, which means it has a flat bow. It is typically 7 feet 9 inches (2.36 meters) long, making it compact and manageable for children. 2. **Simplicity**: The design is straightforward, with a single sail and a simple rigging system, which makes it ideal for beginners to learn the basics of sailing. 3. **Stability**: The flat-bottomed hull provides excellent stability, which is crucial for young sailors who are just starting to learn how to balance and control a sailboat. 4. **Lightweight**: The boat is lightweight, usually around 77 pounds (35 kilograms), making it easy to handle both on and off the water. 5. **Durability**: Optimists are typically made from fiberglass or polyethylene, materials known for their durability and resistance to the elements, ensuring the boat can withstand the rigors of training and racing. 6. **International Class**: The Optimist is recognized as an international class by World Sailing, and it is one of the most popular sailboats for youth racing worldwide, with a strong presence in competitive sailing events. 7. **Training and Racing**: It is widely used for training young sailors and is often the first step in a racing career, with many sailing programs and clubs offering Optimist classes and competitions. These features make the Optimist an excellent choice for introducing children to the sport of sailing.","The Finn is a single-handed, cat-rigged sailboat that is primarily used for racing. It was designed by Rickard Sarby in 1949 and has been an Olympic class since 1952, although it was removed from the Olympic program after the 2020 Tokyo Games. Here are some key features of the Finn: 1. **Design and Construction**: The Finn has a robust and sturdy design, typically made from fiberglass or a combination of fiberglass and carbon fiber. This makes it durable and suitable for various weather conditions. 2. **Hull**: The hull is approximately 4.5 meters (14.8 feet) in length, with a beam of about 1.5 meters (4.9 feet). The hull design allows for excellent maneuverability and stability. 3. **Rigging**: The Finn is equipped with a single sail, a large fully-battened mainsail, which is controlled by a sophisticated rigging system. The sail is typically ma

## Running a question with scenarios
If we instead want to add the scenarios to the question when it is run, we simply add them with the `by()` method. This will re-administer a question for each scenario:

In [7]:
results = q.by(s).run()

The results now include columns for the single question but with a separate row for each scenario:

In [8]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.features
4,cache_keys.features_cache_key
5,cache_used.features_cache_used
6,comment.features_comment
7,generated_tokens.features_generated_tokens
8,iteration.iteration
9,model.frequency_penalty


In [9]:
results.select("sailboat_model", "features")  # results.select("scenario.*", "answer.*") is equivalent here

,scenario.sailboat_model,answer.features
0,Laser,"The Laser sailboat is a popular single-handed racing dinghy known for its simplicity, performance, and widespread use in competitive sailing. Here are some key features: 1. **Design and Construction**: The Laser has a sleek, lightweight design with a flat hull that enhances speed and maneuverability. It is typically constructed from fiberglass, making it durable and easy to handle both on and off the water. 2. **Rigging**: The Laser uses a single sail rig, which is simple and efficient. It features a sleeved sail that fits over the mast, and the rigging is minimal, making it easy to set up and sail. 3. **Sail Options**: The Laser offers different sail sizes to accommodate various skill levels and conditions, including the Standard, Radial, and 4.7 rigs. This versatility makes it suitable for a wide range of sailors, from beginners to advanced racers. 4. **Performance**: The Laser is renowned for its speed and agility, making it a favorite in competitive sailing. Its responsive handling allows sailors to quickly adjust to changing wind and water conditions. 5. **One-Design Class**: The Laser is a one-design class, meaning all boats are built to the same specifications. This ensures a level playing field in races, where skill and tactics are the primary determinants of success. 6. **Popularity and Accessibility**: The Laser is one of the most popular sailboats globally, with a strong presence in both recreational and competitive sailing communities. Its simplicity and affordability make it accessible to a broad range of sailors. 7. **Olympic Class**: The Laser is an Olympic class boat, featured in both men's and women's sailing events, further cementing its status as a premier racing dinghy."
1,Sunfish,"The Sunfish is a popular and iconic sailboat known for its simplicity and ease of use, making it ideal for beginners and recreational sailing. Key features of the Sunfish include: 1. **Design**: The Sunfish has a distinctive, lightweight, and flat-bottomed hull design, which contributes to its stability and ease of handling. It typically measures about 13.9 feet in length and has a beam (width) of around 4.1 feet. 2. **Rigging**: It features a lateen rig, which comprises a single, triangular sail mounted on a simple, two-piece mast and boom. The lateen sail is easy to rig and adjust, making it accessible for novice sailors. 3. **Construction**: Traditionally, Sunfish sailboats were made of fiberglass, which provides durability and low maintenance. The hull is designed to be unsinkable, with foam flotation built in. 4. **Portability**: The boat is lightweight, generally around 120 pounds, which makes it easy to transport on a trailer or car top and launch from a beach or dock. 5. **Performance**: While designed for simplicity, the Sunfish can be quite fast and responsive, offering a fun sailing experience. It performs well in a variety of wind conditions. 6. **Popularity**: Due to its affordability, ease of use, and widespread availability, the Sunfish is one of the most popular sailboats in the world, with a strong community and numerous racing events dedicated to it. Overall, the Sunfish is celebrated for its versatility, making it a great choice for both casual sailing and competitive racing."
2,Optimist,"The Optimist sailboat is a small, single-handed dinghy designed specifically for young sailors. Here are some of its key features: 1. **Size and Design**: The Optimist is a pram dinghy, which means it has a flat bow. It is typically 7 feet 9 inches (2.36 meters) long, making it compact and manageable for children. 2. **Simplicity**: The design is straightforward, with a single sail and a simple rigging system, which makes it ideal for beginners to learn the basics of sailing. 3. **Stability**: The flat-bottomed hull provides excellent stability, which is crucial for young sailors who are just starting to learn how to balance and control a sailboat. 4. **Lightweight**: The boat 

## Posting to Coop
[Coop](https://www.expectedparrot.com) is a platform for creating, storing and sharing LLM-based research and validating it with human respondents. 
We can post surveys, agents, results and notebooks, such as this one. [Learn more about using Coop](https://docs.expectedparrot.com/en/latest/coop.html).

In [ ]:
from edsl import Notebook

nb = Notebook(path = "question_loop_scenarios.ipynb")

nb.push(
    description = "Question method `loop` for creating questions with scenarios", 
    alias = "question-loop-scenarios",
    visibility = "public"
)